<a href="https://colab.research.google.com/github/aboagyeso/v1_code/blob/master/Copy_of_Copy_of_Q05397_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Embedding, Conv1D, MaxPooling1D, GRU
from keras import regularizers
from keras.optimizers import SGD
from keras.optimizers import Adam, RMSprop
from keras.layers import LSTM
import numpy
import csv
import pandas as pd
import hashlib
import random 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

seed = 7
numpy.random.seed(seed)

maxlen = 100
batch_size = 128

dataset = pd.read_csv("https://raw.githubusercontent.com/aboagyeso/LSTM_SMILES/master/Data/smiles_training/v1/Q05397.csv", delimiter=",")
#X_train = dataset.iloc[:,0:1].values
#y_train = dataset.iloc[:,1:2].values
X_train = dataset[['smiles']].values
y_train = dataset[['pAc']].values

for p in range (X_train.shape[0]):
  s = X_train[p,0]
  s = s.replace("[nH]","A")
  s = s.replace("Cl","L")
  s = s.replace("Br","R")
  s = s.replace("[C@]","C")
  s = s.replace("[C@@]","C")
  s = s.replace("[C@@H]","C")
  s =[s[i:i+1] for i in range(0,len(s),1)]
  s = " ".join(s)
  X_train[p,0] = s
X_train = X_train[:,0]  
#y_train = y_train[:,0]
X_train = X_train.tolist()


tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=100)
print(X_train)


[[0 0 0 ... 1 1 4]
 [0 0 0 ... 6 1 5]
 [0 0 0 ... 1 2 5]
 ...
 [0 0 0 ... 1 1 5]
 [0 0 0 ... 1 1 5]
 [0 0 0 ... 3 1 1]]


In [0]:
dataset = pd.read_csv("https://raw.githubusercontent.com/aboagyeso/LSTM_SMILES/master/Data/smiles_validation/smilesQ05397.csv", delimiter=",")
#X_test = dataset.iloc[:,0:1].values
#y_test = dataset.iloc[:,1:2].values

X_test = dataset[['smiles']].values
y_test = dataset[['pAc']].values




#X_test = dataset.iloc[:,0:1].values
#y_test = dataset.iloc[:,1:2].values

for p in range (X_test.shape[0]):
  s = X_test[p,0]
  s = s.replace("[nH]","A")
  s = s.replace("Cl","L")
  s = s.replace("Br","R")
  s = s.replace("[C@]","C")
  s = s.replace("[C@@]","C")
  s = s.replace("[C@@H]","C")
  s =[s[i:i+1] for i in range(0,len(s),1)]
  s = " ".join(s)
  X_test[p,0] = s
X_test = X_test[:,0]  
#y_test = y_test[:,0]
X_test = X_test.tolist()
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=100)
#print(X_test)
#print(X_test.shape)



model = Sequential()
model.add(Embedding(100, 128, input_length=100))
model.add(LSTM(128))
model.add(Dense(100, activation='tanh'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer=Adam(0.001))


model.fit(X_train, y_train,
          batch_size=128,
          epochs=20)



Epoch 1/20
5355/5355 [==============================] - 16s 3ms/step - loss: 11.0117
Epoch 2/20
5355/5355 [==============================] - 15s 3ms/step - loss: 1.6121
Epoch 3/20
5355/5355 [==============================] - 15s 3ms/step - loss: 1.5865
Epoch 4/20
5355/5355 [==============================] - 15s 3ms/step - loss: 1.5778
Epoch 5/20
5355/5355 [==============================] - 15s 3ms/step - loss: 1.5626
Epoch 6/20
5355/5355 [==============================] - 15s 3ms/step - loss: 1.5529
Epoch 7/20
5355/5355 [==============================] - 15s 3ms/step - loss: 1.5287
Epoch 8/20
5355/5355 [==============================] - 15s 3ms/step - loss: 1.5133
Epoch 9/20
5355/5355 [==============================] - 15s 3ms/step - loss: 1.5022
Epoch 10/20
5355/5355 [==============================] - 15s 3ms/step - loss: 1.4813
Epoch 11/20
5355/5355 [==============================] - 15s 3ms/step - loss: 1.4497
Epoch 12/20
5355/5355 [==============================] - 15s 3ms/step - l

In [0]:
score = model.evaluate(X_test, y_test,
                            batch_size=128)
print('Test score:', score)

predictions = model.predict(X_test)


mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2_score = r2_score(y_test, predictions)

print(str(mae)+"\t"+str(mse)+"\t"+str(r2_score))

103/103 [==============================] - 0s 3ms/step
Test score: 0.935662031173706
0.7932858804128701	0.935661944771541	0.40579715388637394
